## Exercise 7 | data in f1ex7 | Calculate the percentage of individuals living within 500 meters of a recycling bin in each “Freguesia” of Coimbra.

In [ ]:
units_shp = '/mnt/d/cwsig/exercicios_2023_24/f1ex7/freg_coimbra.shp'
units_pk  = 'DICOFRE'

pop_shp = '/mnt/d/cwsig/exercicios_2023_24/f1ex7/bgri_coimbra.shp'
units_fk = 'idfreg'
popcol = 'popres21'

pnt_shp = '/mnt/d/cwsig/exercicios_2023_24/f1ex7/ecopontos19.shp'

outcol = 'popp'

bfdist = 500

In [ ]:
import pandas as pd
import geopandas as gp
from glass.rd.shp import shp_to_obj
from glass.prop.prj import get_epsg

from glass.prj.obj import df_prj
from glass.gp.prox.bfing.obj import gs_buffer
from glass.dtt.cg.obj import centroid_dfgeoms
from glass.gp.ovl.obj import count_pnt_inside_poly

In [ ]:
d = {
    'units' : units_shp,
    'pop'   : pop_shp,
    'pnt'   : pnt_shp
}

In [ ]:
epsgs = {}
for k in d:
    epsgs[k] = get_epsg(d[k])
    d[k] = shp_to_obj(d[k])

In [ ]:
if epsgs['units'] != epsgs['pop']:
    d['pop'] = df_prj(d['pop'], epsgs['units'])

if epsgs['units'] != epsgs['pnt']:
    d['pnt'] = df_prj(d['pnt'], epsgs['units'])

In [ ]:
# Get Population Centroids
d['pop'] = centroid_dfgeoms(d['pop'], 'geometry', epsg=epsgs['units'])

# Go for buffer
d['pop']['geometry'] = gs_buffer(d['pop']['geometry'], bfdist)

In [ ]:
d['pop'] = count_pnt_inside_poly(d['pnt'], 'countpnt', d['pop'])

pop_w5 = d['pop'][d['pop']['countpnt'] > 0]

In [ ]:
# Group By to get population within 500 metros
popw5gp = pd.DataFrame(pop_w5.groupby([units_fk]).agg({
    popcol: 'sum'
})).reset_index()

# Get Total population
popunits = pd.DataFrame(d['pop'].groupby([units_fk]).agg({
    popcol: 'sum'
})).reset_index()

In [ ]:
# Join data with main dataframe and calculate percentage
d['units'] = d['units'].merge(
    popunits, how='left', 
    left_on=units_pk, right_on=units_fk
)

d['units'].drop(units_fk, axis=1, inplace=True)
d['units'].rename(columns={popcol: 'total_pop'}, inplace=True)

d['units'] = d['units'].merge(
    popw5gp, how='left', 
    left_on=units_pk, right_on=units_fk
)

d['units'].rename(columns={popcol: 'popwithin'}, inplace=True)

d['units'][outcol] = d['units'].popwithin / d['units'].total_pop * 100 

In [ ]:
d['units']